In [1]:
from HPC_classes import *
import warnings
warnings.filterwarnings("ignore")
df , _ = SparkDataProcessor().process_data(
    data_csv_url= "https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/data/data.csv", 
    drop_col_yml_url= "https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/params.yml", 
    rename_col_json_url="https://raw.githubusercontent.com/DeepenData/Phenylketonuria_IR_and_Machine_Learning/alejandro/labels_relabel.json")

23/05/28 23:26:56 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/05/28 23:26:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/28 23:26:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/28 23:26:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
my_target               = 'HOMA-IR altered'
it_targets  : list[str] = [my_target]
it_features : list[str] = ['Phe']

#df.drop(my_target, axis=1, inplace=True)

In [3]:
ray_data = ray.put(df)

2023-05-28 23:27:01,199	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [4]:
%%time

studies = make_multiple_studies(
    ray_data, 
    features = it_features,
    targets = it_targets,
    n_trials=1000
)

studies = ray.get(studies)

(make_a_study pid=3667579) [I 2023-05-28 23:27:02,596] A new study created in memory with name: Phe_predicts_HOMA-IR altered
(make_a_study pid=3667579) [I 2023-05-28 23:27:03,479] Trial 0 finished with values: [0.5, 0.0] and parameters: {'max_depth': 5, 'eta': 0.11419417268021187, 'subsample': 0.17043921085413696, 'lambda': 0.590648559043604, 'alpha': 0.7760223207559017, 'scale_pos_weight': 2.930990057175842, 'kfold_splits': 2}. 
(make_a_study pid=3667579) [I 2023-05-28 23:27:03,595] Trial 7 finished with values: [0.5, 0.0] and parameters: {'max_depth': 10, 'eta': 0.18430404962842828, 'subsample': 0.184498304116956, 'lambda': 0.5326103412536639, 'alpha': 0.8815912628452905, 'scale_pos_weight': 1.9128690553785654, 'kfold_splits': 2}. 
(make_a_study pid=3667579) [I 2023-05-28 23:27:03,647] Trial 1 finished with values: [0.5961538461538461, 0.0] and parameters: {'max_depth': 7, 'eta': 0.4836687272344889, 'subsample': 0.374783441132151, 'lambda': 0.15251271465472283, 'alpha': 0.00880503603

CPU times: user 3.23 s, sys: 2.29 s, total: 5.51 s
Wall time: 2min 57s


(make_a_study pid=3667579) [I 2023-05-28 23:29:59,643] Trial 998 finished with values: [0.5694444444444444, 0.0] and parameters: {'max_depth': 7, 'eta': 0.49471488766797533, 'subsample': 0.25794934672077013, 'lambda': 0.09354155183759716, 'alpha': 0.22504986697214413, 'scale_pos_weight': 4.3782436173592005, 'kfold_splits': 3}. 
(make_a_study pid=3667579) [I 2023-05-28 23:29:59,675] Trial 999 finished with values: [0.7777777777777778, 0.0] and parameters: {'max_depth': 9, 'eta': 0.49471488766797533, 'subsample': 0.39861202202813584, 'lambda': 0.3052623182554641, 'alpha': 0.48452831641395944, 'scale_pos_weight': 4.966128529461416, 'kfold_splits': 4}. 


In [5]:

make_pareto_plot(studies[0])

In [6]:
a_multi_study  = studies[0]
CUTOFF_AUC     = 0.9
CUTOFF_FEATURE = 0



bs_trials = []#pd.DataFrame()
ds_aucs   = []
for t in a_multi_study.get_trials():
        if t.values is None:
                continue
        if t.values[0] > CUTOFF_AUC and t.values[1] > CUTOFF_FEATURE:
                #bs_trials[t.number] = t.params#list(t.params.values()) #+ list(t.values)
                bs_trials.append(t.params)
                ds_aucs.append(t.values[0])



df_ranks_abnormal = pd.DataFrame()
df_ranks_healty   = pd.DataFrame()
AUCs              = []


for trial_num, one_trial in enumerate(bs_trials):


    kfold_splits, xgb_params = one_trial.pop('kfold_splits'), one_trial

    model_instance = ModelInstance(
                                        df  = df, 
                                        target      = it_targets[0], 
                                        test_size = 0.3, 
                                        xgb_params  = xgb_params, 
                                        kfold_splits = kfold_splits, 
                                        seed       = None)

    feature_metrics = model_instance.get_feature_explanation()
    df_ranks_abnormal[trial_num] = feature_metrics["SHAP_abnormal"]
    df_ranks_healty[trial_num]   = feature_metrics["SHAP_healty"]
    AUCs.append(model_instance.get_AUC_on_test_data())


import plotly.express as px

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index

df_ranks_long["auc_score"] = AUCs


fig_barplot = px.bar(
df_ranks_abnormal,
barmode="group",
)
def rank_sort(df_ranks_abnormal : pd.DataFrame) -> pd.Index:
        """This takes the dataframe, ranks its variables, and sorts them by the mean ranking"""
        return df_ranks_abnormal.rank(axis="rows").mean(axis="columns").sort_values(ascending=False).index

import plotly.express as px
order = rank_sort(df_ranks_abnormal) # Converts ranks to an ordered list

def wrapper_pcoorplot(df_ranks_abnormal, dims = order.tolist()):
        """Small wapper around plotly paralel coordinates plot"""
        fig_ranks = px.parallel_coordinates(
                df_ranks_abnormal.T, 
                dimensions=dims,
                color="Phe",
                #labels=labels_relabel,
                color_continuous_scale='BuGn',
        )

        fig_ranks.update_layout(
                yaxis_tickformat=".2f",
                coloraxis_colorbar=dict(title=f"{a_multi_study.study_name} (Shapley value)"),
        )

        return fig_ranks

# Creates the figure with the first [:15] features according to ranking
fig_abnormal = wrapper_pcoorplot(df_ranks_abnormal, order.tolist()[:10])
fig_abnormal.show() # Show fig, as this is a notebook